In [ ]:
import pandas as pd
import torch
from sentence_transformers import SentenceTransformer, util

In [ ]:
celulares_df = pd.read_csv("/home/caua/GoogleDrive/ENGENHARIA/COMPUTAÇÃO/MATÉRIAS SEMESTRE 2/PROJETOS 2/AMBIENTE/dataset_celulares_final.csv")

In [ ]:
# --- CÉLULA DE FILTRO (HARD FILTERS) ---

# 1. Criar uma cópia para preservar o original
banco_filtravel = celulares_df.copy()

# === NOVA LINHA: LIMPEZA DE DADOS ===
# Remove aparelhos com preço zerado ou negativo
banco_filtravel = banco_filtravel[banco_filtravel['Preco_Num'] > 0]

print(f"Total inicial de aparelhos (com preço válido): {len(banco_filtravel)}")

# 2. Filtro de Preço (Obrigatório)
if user_choices.get("orcamento_max", 0) > 0:
    banco_filtravel = banco_filtravel[banco_filtravel['Preco_Num'] <= user_choices['orcamento_max']]

# 3. Filtro de Marca (Opcional)
if user_choices.get("marcas_preferidas"):
    banco_filtravel = banco_filtravel[banco_filtravel['Marca'].isin(user_choices['marcas_preferidas'])]

# 4. Filtro de RAM (Requisito Mínimo)
if user_choices.get("target_ram", 0) > 0:
    banco_filtravel = banco_filtravel[banco_filtravel['RAM_Num'] >= user_choices['target_ram']]

# 5. Filtro de Armazenamento (Requisito Mínimo)
if user_choices.get("target_armazenamento", 0) > 0:
    banco_filtravel = banco_filtravel[banco_filtravel['Armazenamento_Num'] >= user_choices['target_armazenamento']]

# 6. Filtro de Bateria (Requisito Mínimo)
if user_choices.get("target_bateria", 0) > 0:
    banco_filtravel = banco_filtravel[banco_filtravel['Bateria_Num'] >= user_choices['target_bateria']]

# Resetar o index para organizar a tabela final
banco_filtravel = banco_filtravel.reset_index(drop=True)

# --- RESULTADO DO FUNIL ---
print(f"Total final após filtros: {len(banco_filtravel)}")
display(banco_filtravel[['Nome', 'Preco_Num', 'Marca', 'RAM_Num', 'Armazenamento_Num']].head())

In [ ]:
# --- PEÇA 1: O MODELO (O TRADUTOR) ---
model = SentenceTransformer('all-MiniLM-L6-v2') 

# Aqui a IA trabalha. Ela transforma texto em vetor.
vetor_banco = model.encode(banco_filtravel['Specs_Completa']) 
vetor_usuario = model.encode(query)

# --- PEÇA 2: O ALGORITMO (O MATEMÁTICO) ---
# Aqui NÃO TEM IA. É pura álgebra linear (multiplicação de matrizes).
# 'util.cos_sim' é a função que calcula o ângulo.
scores = util.cos_sim(vetor_usuario, vetor_banco)

In [ ]:
# 1. Obter os 3 maiores scores e seus índices
# scores[0] garante que estamos pegando a linha da sua query única
# k=3 é o número de resultados que você quer
vals, indices = torch.topk(scores, k=3) 

# 2. Converter os tensores do PyTorch para listas simples do Python
indices_lista = indices[0].tolist() # Pega os índices (ex: [10, 5, 42])
scores_lista = vals[0].tolist()     # Pega as notas (ex: [0.92, 0.88, 0.85])

# 3. Selecionar as linhas correspondentes no banco filtrado
top_3_celulares = banco_filtravel.iloc[indices_lista].copy()

# 4. (Opcional) Adicionar a nota de similaridade na tabela para você ver
# Isso ajuda muito a entender por que o modelo escolheu eles
top_3_celulares['Match_Score'] = scores_lista

# 5. Exibir o resultado formatado
display(top_3_celulares[['Nome', 'Marca', 'Preco_Num', 'Match_Score', 'RAM_Num', 'Armazenamento_Num']])